In [4]:
import pandas as pd

In [5]:
df_movies = pd.read_csv("Movies_all_ML.csv")
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movieId               87585 non-null  int64  
 1   imdbId                87585 non-null  object 
 2   vote_average          86493 non-null  float64
 3   vote_count            86493 non-null  float64
 4   status                86493 non-null  object 
 5   release_date          87455 non-null  object 
 6   revenue               86493 non-null  float64
 7   runtime               86493 non-null  float64
 8   adult                 86493 non-null  object 
 9   backdrop_path         70442 non-null  object 
 10  budget                86493 non-null  float64
 11  homepage              17774 non-null  object 
 12  original_language     86493 non-null  object 
 13  original_title        86493 non-null  object 
 14  overview              85667 non-null  object 
 15  popularity         

In [6]:
df_movies.head()

,movieId,imdbId,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,production_companies,production_countries,spoken_languages,keywords,title,genres
0,1,tt0114709,7.971,17152.0,Released,1995-10-30,394400000.0,81.0,False,/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg,...,"Led by Woody, Andy's toys live happily in his ...",78.404,/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,Hang on for the comedy that goes to infinity a...,Pixar,United States of America,English,"rescue, friendship, mission, martial arts, jea...",Toy Story,"Animation, Adventure, Family, Comedy"
1,2,tt0113497,7.239,9833.0,Released,1995-12-15,262821940.0,104.0,False,/pYw10zrqfkdm3yD9JTO6vEGQhKy.jpg,...,When siblings Judy and Peter discover an encha...,13.444,/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg,Roll the dice and unleash the excitement!,"TriStar Pictures, Interscope Communications, T...",United States of America,"English, French","giant insect, board game, disappearance, jungl...",Jumanji,"Adventure, Fantasy, Family"
2,3,tt0113228,6.476,347.0,Released,1995-12-22,71500000.0,101.0,False,/1J4Z7VhdAgtdd97nCxY7dcBpjGT.jpg,...,A family wedding reignites the ancient feud be...,14.815,/1FSXpj5e8l4KH6nVFO5SPUeraOt.jpg,Still Yelling. Still Fighting. Still Ready for...,"Lancaster Gate, Warner Bros. Pictures",United States of America,English,"fishing, sequel, old man, best friend, wedding...",Grumpier Old Men,"Romance, Comedy"
3,4,tt0114885,6.183,142.0,Released,1995-12-22,81452156.0,127.0,False,/jZjoEKXMTDoZAGdkjhAdJaKtXSN.jpg,...,"Cheated on, mistreated and stepped on, the wom...",14.451,/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg,Friends are the people who let you be yourself...,20th Century Fox,United States of America,English,"based on novel or book, interracial relationsh...",Waiting to Exhale,"Comedy, Drama, Romance"
4,5,tt0113041,6.228,659.0,Released,1995-12-08,76594107.0,106.0,False,/lEsjVrGU21BeJjF5AF9EWsihDpw.jpg,...,Just when George Banks has recovered from his ...,14.537,/rj4LBtwQ0uGrpBnCELr716Qo3mw.jpg,Just when his world is back to normal... he's ...,"Touchstone Pictures, Sandollar Productions",United States of America,English,"daughter, baby, parent child relationship, mid...",Father of the Bride Part II,"Comedy, Family"


In [7]:
df_crew = pd.read_csv(
    "Crew_dataset.gz",
    sep="\t",
    na_values="\\N",
    usecols=["tconst", "directors"]
)

In [8]:
df_name = pd.read_csv(
    "Basic_names_crew_dataset.gz",
    sep="\t",
    na_values="\\N",
    usecols=["nconst", "primaryName"]
)

In [9]:
df_principals = pd.read_csv(
    "title.principals.tsv.gz",
    sep="\t",
    na_values="\\N",
    usecols=["tconst", "ordering", "nconst", "category"]
)

In [10]:
df_crew = df_crew.dropna(subset=["directors"]).copy()
df_crew["directors"] = df_crew["directors"].str.split(",")

df_directors = df_crew.explode("directors").rename(columns={"directors": "nconst"})
df_directors = df_directors.merge(df_name, on="nconst", how="inner")

df_directors_grouped = (
    df_directors
    .groupby("tconst", as_index=False)
    .agg({"primaryName": lambda x: ", ".join(x.dropna().unique())})
    .rename(columns={"primaryName": "Directors"})
)

In [11]:
# Szukamy aktorów i aktorek
df_cast = df_principals[
    df_principals["category"].isin(["actor", "actress"])
].copy()

df_cast = df_cast.merge(df_name, on="nconst", how="inner")

df_cast_grouped = (
    df_cast
    .groupby("tconst", as_index=False)
    .agg({"primaryName": lambda x: ", ".join(x.dropna().unique())})
    .rename(columns={"primaryName": "Cast"})
)

In [12]:
df_cast["ordering"] = pd.to_numeric(df_cast["ordering"], errors="coerce")
df_cast.sort_values(["tconst", "ordering"], inplace=True)

# TOP 3 aktorów lub aktorek
df_top3 = df_cast.groupby("tconst").head(3).copy()

df_star_grouped = (
    df_top3
    .groupby("tconst", as_index=False)
    .agg({"primaryName": lambda x: ", ".join(x.dropna().unique())})
    .rename(columns={"primaryName": "StarActors"})
)

In [13]:
df_dir_cast = df_directors_grouped.merge(df_cast_grouped, on="tconst", how="inner")
df_dir_cast_star = df_dir_cast.merge(df_star_grouped, on="tconst", how="inner")

In [14]:
df_final = df_movies.merge(
    df_dir_cast_star,
    left_on="imdbId",
    right_on="tconst",
    how="left"
)

In [15]:
columns_to_drop = ["tconst", "nconst", "category", "ordering"]
for col in columns_to_drop:
    if col in df_final.columns:
        df_final.drop(columns=[col], inplace=True)

In [16]:
df_final.head()
df_final.to_csv("Movies_final_ML.csv", index=False)


In [17]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movieId               87585 non-null  int64  
 1   imdbId                87585 non-null  object 
 2   vote_average          86493 non-null  float64
 3   vote_count            86493 non-null  float64
 4   status                86493 non-null  object 
 5   release_date          87455 non-null  object 
 6   revenue               86493 non-null  float64
 7   runtime               86493 non-null  float64
 8   adult                 86493 non-null  object 
 9   backdrop_path         70442 non-null  object 
 10  budget                86493 non-null  float64
 11  homepage              17774 non-null  object 
 12  original_language     86493 non-null  object 
 13  original_title        86493 non-null  object 
 14  overview              85667 non-null  object 
 15  popularity         